In [20]:
import os
from dotenv import load_dotenv
from langchain_together import ChatTogether

load_dotenv(override=True)

llm = ChatTogether(api_key=os.getenv("TOGETHER_API_KEY"), temperature=0.0, model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")


In [9]:
!pip install pypdf

In [10]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    PyPDFLoader("airlines.pdf")
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

print(len(docs))

25


In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 800, # max size of each chunk in num characters
    chunk_overlap = 200, # overlap between chunk in num characters
    separators=["\n\n", "\n", ".", " "]
)

In [12]:
splits = text_splitter.split_documents(docs)
print(len(splits))

93


In [13]:
!pip install --quiet --upgrade tf-keras
!pip install --quiet --upgrade torch
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade --quiet langchain-core
!pip install --upgrade --quiet langchain
!pip install --upgrade --quiet langchain-community
!pip install --upgrade --quiet langchain-together

In [14]:
!pip install --upgrade transformers

In [15]:
!pip install --upgrade sentence-transformers

In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings()

/var/folders/_6/sw6vhpc10411wn4lc23xd4t80000gn/T/ipykernel_72281/1103018835.py:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()


In [17]:
!pip install faiss-cpu

In [18]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(splits, embedding)

print(db.index.ntotal)

93


In [21]:
from langchain.chains import RetrievalQA

# create a RetrievalQA chain using the specified LLM and FAISS retriever
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever() # FAISS index as the retriever for relevant documents
)

In [23]:
# Test
query = "What all items I cannot place in my check-in luggage?"

answer = qa_chain({"query": query})
print(answer["result"])

Based on the provided text, the following items cannot be placed in check-in luggage:

1. Items specified in the Dangerous Goods Regulations of ICAO and IATA, and Carrier's Regulations.
2. Items prohibited by the applicable laws, regulations, or orders of any state to be flown from, to, or over.
3. Fragile or perishable items.
4. Live animals.
5. Money.
6. Jewelry.
7. Precious metals.
8. Silverware.
9. Negotiable papers.
10. Securities.
11. Business documents.
12. Passports.
13. Other identification documents.
14. Samples.
15. Weapons such as antique firearms, swords, knives, and arms and ammunition (licensed revolver).
16. Firearms and ammunition for hunting and sporting activities.
17. Delicate musical instruments (these can be carried in the cabin compartment if approved by the Carrier).
